<a href="https://colab.research.google.com/github/maryamteimouri/football_player_detection/blob/main/VisionFinalProj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x
import cv2
import numpy as np
import tensorflow as tf
from PIL import Image

from keras.datasets import mnist
from keras.layers import Input ,Dense,Activation, Conv2D,AveragePooling2D,Flatten
from keras.models import Model
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
get_ipython().magic(u'matplotlib inline')
import matplotlib.pyplot as plt
# import tensorflow_datasets as tfds

from tensorflow.keras import layers
from google.colab.patches import cv2_imshow
import os

In [ ]:
def make_bird_eye_view(src):
    
    img_h = 750
    img_w = 1050
    
    points1 = np.array([(25, 145),
                        (1225, 89),
                        (675, 200),
                        (870, 780)]).astype(np.float32)
    
    points2 = np.array([(0,0),
                        (img_w,0),
                        (img_w/2,img_h/2),
                        (img_w/2,img_h)]).astype(np.float32)
    
    H = cv2.getPerspectiveTransform(points1, points2)
    src = np.append(np.asarray(src),1)
    t = np.dot(H, src)
    t = t / t[2]
    return t
  

In [ ]:
def detect(frame,fgmask):
    th = cv2.threshold(fgmask, 250, 255, cv2.THRESH_BINARY)[1]
    dilated = cv2.dilate(th, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (1,1)), iterations = 2)
    contours, hier = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    return contours

In [ ]:
def load_data():
  arr = os.listdir('/content/drive/MyDrive/Colab Notebooks/blue_resized')
  arr2 = os.listdir('/content/drive/MyDrive/Colab Notebooks/red_resized')
  train_y=np.array([])
  train_x=np.array([[[]]])
  # counter = 0
  for element in arr:
    path='/content/drive/MyDrive/Colab Notebooks/blue_resized/'
    path+=element
    I1 = cv2.imread(path)
    # print(I1.shape)
    I1 = np.array(I1)
    train_y = np.append(train_y, 1);
    train_x = np.append(train_x,I1)
    # if counter > 10:
    #   break
    # counter+=1
  # counter=0
  for element in arr2:
    path='/content/drive/MyDrive/Colab Notebooks/red_resized/'
    path+=element
    I1 = np.array(I1)
    train_y=np.append(train_y,0);
    train_x=np.append(train_x,I1)
    # if counter > 10:
    #   break
    # counter+=1

  return train_x, train_y

In [ ]:
def build_model(input_shape):
  
  x_input = Input(shape =input_shape,name = 'input')
  x = Conv2D(filters = 16,kernel_size = (2,2), strides = 1, padding = 'valid',name = 'conv2')(x_input)
  x = Activation('relu')(x)
  x = AveragePooling2D(pool_size =2,strides = 2,name = 'pad2')(x)

  x = Flatten()(x)

  x = Dense(units = 120, name = 'fc_1')(x)

  x = Activation('relu', name = 'relu_1')(x)
  # x = Dropout(rate = 0.5)

  x = Dense(units = 84, name = 'fc_2')(x)
  x = Activation('relu', name = 'relu_2')(x)
  # x = Dropout(rate = 0.5)


  outputs = Dense(units = 2,name = 'softmax', activation='softmax')(x)
  
  model = Model(inputs = x_input, outputs = outputs)
  model.summary()

  return model

In [ ]:
def train():
  x_train, y_train = load_data()
  x_train = x_train.reshape((-1,195,192,3))
  x_train, x_test, y_train, y_test  = train_test_split(x_train, y_train , test_size=0.1, shuffle=True, random_state=5)
    # image_size = train_x.shape[1]
  x_train = x_train.astype('float32')/255
  x_test = x_test.astype('float32')/255
  # print(x_train.shape)
  
  x_test = x_test.reshape((-1,195,192,3))
  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test)
  model = build_model(input_shape=(195,192,3))
  model.compile(optimizer = 'adam',loss = 'categorical_crossentropy' ,metrics = ['accuracy'])
  # model.fit(x = x_train, y= y_train , batch_size=5, epochs = 20)
  datagen = ImageDataGenerator(
          rotation_range=30,
          width_shift_range=0.2,
          height_shift_range=0.2,
          zoom_range=0.2,
          horizontal_flip=True,
          fill_mode='nearest')
  batch_size = 64

 
  H = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
  validation_data=(x_test, y_test),                       
	steps_per_epoch=len(y_train) // batch_size, epochs=10)
  return model, x_test, y_test


In [ ]:
def test(mode, test_x, test_y):
  sample = x_test[100]

  sample_show = sample.reshape((200,200))
  plt.figure(figsize=(3,3))
  plt.imshow(sample_show , cmap='gray')
  plt.show()
  batch = np.expand_dims(sample, axis=0)
  prediction = model.predict(batch)
  print(np.argmax(prediction[0]))

In [ ]:
def clasify(image, model):
  # set size
  img_h = 200
  img_w = 200
  points2 = np.array([(0, 0),
                    (img_w, 0),
                    (0, img_h ),
                    (img_w, img_h)]).astype(np.float32)

  points1 = np.array([(0, 0),
                      (image.shape[1], 0),
                      (0, image.shape[0] ),
                      (image.shape[1], image.shape[0])]).astype(np.float32)
  H = cv2.getPerspectiveTransform(points1, points2)
  image = cv2.warpPerspective(image, H, (img_w - 8, img_h - 5))
  batch = np.expand_dims(image, axis=0)
  prediction = model.predict(batch)
  # print(np.argmax(prediction[0]))
  return np.argmax(prediction[0])


In [ ]:
def main():
  fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()
  cap = cv2.VideoCapture('/content/drive/MyDrive/Colab Notebooks/output.h264')
  patch = cv2.imread('/content/drive/MyDrive/Colab Notebooks/visionProjectFiles/2D_field.png')
  patch_copy = patch.copy()
  model, _, _ = train()
  
  while (cap.isOpened()):
    ret, frame = cap.read()
    fgmask = fgbg.apply(frame,5)
    contours = detect(frame,fgmask)
    patch = patch_copy.copy()
    for c in contours:
        if cv2.contourArea(c) > 100:
            (x,y,w,h) = cv2.boundingRect(c)
            cv2.rectangle(frame, (x,y), (x+w,y+h), (255,255,0), 2)
            point = (x+int(w/2),y+h)
            point = make_bird_eye_view(point)

            cropped = frame[y + 2:y + h - 2, x + 2:x + w - 2]
            team_class = clasify(cropped,model)

            if team_class == 0:
              cv2.circle(patch, (int(point[0]), int(point[1])), 1, (0, 0, 255), 10)
            else:
              cv2.circle(patch, (int(point[0]), int(point[1])), 1, (255, 0, 0), 10)

            # height, width, channels = frame.shape

            # size = (width, height)

            # result = cv2.VideoWriter('/content/drive/MyDrive/Colab Notebooks/visionProjectFiles/filename2.avi', 
            #              cv2.VideoWriter_fourcc(*'MJPG'),
            #              10, size)
            # height, width, channels = patch.shape

            # size = (width, height)
            # result2 = cv2.VideoWriter('/content/drive/MyDrive/Colab Notebooks/visionProjectFiles/filename3.avi', 
            #              cv2.VideoWriter_fourcc(*'MJPG'),
            #              10, size)
            
    
            # result.write(frame)
            # result2.write(patch)
                
            cv2_imshow(frame)
            cv2_imshow(patch)

    if cv2. waitKey(1) & 0xFF == ord('q'):
          break
  cap.release()
  cv2.destroyAllWindows()
  return


main()

In [ ]:

path1 = '/content/drive/MyDrive/Colab Notebooks/visionProjectFiles/filename2.avi'

path2 = '/content/drive/MyDrive/Colab Notebooks/visionProjectFiles/filename3.avi'

cap1 = cv2.VideoCapture(path1)
cap2 = cv2.VideoCapture(path2)


while True:

    ok, f1 = cap1.read()
    ok, f2 = cap2.read()

    cv2.imshow('1', f1)
    cv2.imshow('2', f2)
    cv2.waitKey(300)